In [1]:
import pandas as pd
from openpyxl import load_workbook
import openpyxl
import numpy as np
from datetime import date, timedelta

In [2]:
def cell_to_num(char):
    char = char.upper()
    if char.isalpha():
        num = ord(char) - 64
    else:
        num = 0
    return num
print(cell_to_num('a'))

1


In [3]:
# Windows file system
first_file = 'C:/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = 'C:/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'
"""
# linux file system
first_file = '/mnt/c/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = '/mnt/c/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'
"""
wb_check = load_workbook(first_file)

c:\users\panka\appdata\local\programs\python\python38-32\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
def data_cleaning(path):
    wb = load_workbook(path)
    ws = wb.active
    
    #Unmerge cell
    ws.unmerge_cells('O1:P1')
    ws.unmerge_cells('Q1:R1')
    ws.unmerge_cells('S1:T1')
    ws.unmerge_cells('U1:V1')
    if path[-7] == '1':
        #renaming issue_sale value with dates
        ws['S1'] = str(date.today() - timedelta(days=4))
        ws['T1'] = str(date.today() - timedelta(days=3))
    else:
        ws['S1'] = str(date.today() - timedelta(days=2))    
        ws['T1'] = str(date.today() - timedelta(days=1))
    
    print(ws['S1'].value)
    print(ws['T1'].value)    
    
    #Delete column U:V
    ws.delete_cols(cell_to_num('U'), cell_to_num('V')-cell_to_num('U')+1)
    #Delete column O:R
    ws.delete_cols(cell_to_num('O'), cell_to_num('R')-cell_to_num('O')+1)
    #Delete column K:M
    ws.delete_cols(cell_to_num('K'), cell_to_num('M')-cell_to_num('K')+1)
    #Delete column B:F
    ws.delete_cols(cell_to_num('B'), cell_to_num('F')-cell_to_num('B')+1)
    #Delete second row
    ws.delete_rows(2)
    wb.save(path)

In [5]:
def create_pivot(path):
    #reading excel file using pandas
    old_df = pd.read_excel(path)
    current_wb = load_workbook(path)
    ws = current_wb.active
    
    #filter out the 'NA' value in the product column
    df = old_df[
        (old_df['Product'] == 'XP') |
        (old_df['Product'] == 'HS') |
        (old_df['Product'] == 'MS')
    ]
    
    df.loc[df['Product'] == 'XP', 'Product'] = 'MS'
    
    pvt_all = pd.pivot_table(df,
                            index=["SA"],
                            columns=['Product'],
                            values=[ws['H1'].value, ws['G1'].value],
                            aggfunc=np.sum)
    return pvt_all

In [6]:
data_cleaning(first_file)
data_cleaning(second_file)

2020-08-18
2020-08-19
2020-08-16
2020-08-17


In [7]:
pvt = pd.concat([create_pivot(first_file), create_pivot(second_file)], axis=1)
print(pvt)

c:\users\panka\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


                     2020-08-18           2020-08-19           2020-08-16  \
Product                      HS        MS         HS        MS         HS   
SA                                                                          
Fuels- Bargarh(R)      38821.91  48629.88   48173.73  49982.35   32889.11   
Fuels-Bhawanipatna(R   47192.44  53251.30   42687.11  44234.87   39575.28   
Fuels-JEYPORE(R)      105962.00  68978.65   76242.47  50408.61   50165.91   
Fuels-Jharsuguda(R)   126889.83  42844.78  154855.57  42621.40   72423.87   
Fuels-ROURKELA(R)     275982.19  54264.92  274345.39  48917.65  260915.45   
Fuels-SAMBALPUR (R)    72204.61  39106.79   88873.02  38263.36   65449.13   
Keonjhar I RSA         81982.21  24699.31   73249.58  17943.48   81805.33   
Keonjhar II RSA         8533.56   9627.54    8608.11   6954.99    3594.88   

                               2020-08-17            
Product                     MS         HS        MS  
SA                                          

In [8]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    pvt.to_excel(writer, 'pivot sheet', index=True)

In [9]:
df = pd.read_excel(first_file, sheet_name='pivot sheet')
print(df.columns.values)

['Unnamed: 0' '2020-08-18' 'Unnamed: 2' '2020-08-19' 'Unnamed: 4'
 '2020-08-16' 'Unnamed: 6' '2020-08-17' 'Unnamed: 8']


In [10]:
cols = df.columns.values
order = [0, 3, 4, 1, 2, 7, 8, 5, 6]
cols = [cols[i] for i in order]
print(cols)

['Unnamed: 0', '2020-08-19', 'Unnamed: 4', '2020-08-18', 'Unnamed: 2', '2020-08-17', 'Unnamed: 8', '2020-08-16', 'Unnamed: 6']


In [11]:
df = df[cols]
print(df)

             Unnamed: 0 2020-08-19 Unnamed: 4 2020-08-18 Unnamed: 2  \
0               Product         HS         MS         HS         MS   
1                    SA        NaN        NaN        NaN        NaN   
2     Fuels- Bargarh(R)    48173.7    49982.4    38821.9    48629.9   
3  Fuels-Bhawanipatna(R    42687.1    44234.9    47192.4    53251.3   
4      Fuels-JEYPORE(R)    76242.5    50408.6     105962    68978.6   
5   Fuels-Jharsuguda(R)     154856    42621.4     126890    42844.8   
6     Fuels-ROURKELA(R)     274345    48917.7     275982    54264.9   
7   Fuels-SAMBALPUR (R)      88873    38263.4    72204.6    39106.8   
8        Keonjhar I RSA    73249.6    17943.5    81982.2    24699.3   
9       Keonjhar II RSA    8608.11    6954.99    8533.56    9627.54   

  2020-08-17 Unnamed: 8 2020-08-16 Unnamed: 6  
0         HS         MS         HS         MS  
1        NaN        NaN        NaN        NaN  
2    43583.9    50826.3    32889.1    25587.9  
3    49073.1    54930.8   

In [12]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    df.to_excel(writer, 'summary', index=False)

In [13]:
wb = load_workbook(first_file)
ws = wb['summary']
ws.delete_rows(3)
check = 0
for value in ws.iter_rows(min_row=1, max_row=1):
    for cell in value:
        if check % 2 == 0:
            cell.value = ""
        check += 1
        print(cell.value)
# print(df.columns.values)
print(ws['A3'].value)


2020-08-19

2020-08-18

2020-08-17

2020-08-16

Fuels- Bargarh(R)


In [14]:
"""
data = 'ABCDEFGHI'
for char in data:
    val = char + '1'
    print(ws[val].value)

print(ws.cell(row=1, column=2).value)

#Iterate through columns of the first row
for row in ws.iter_rows(min_row=1, max_row=1):
    for cell in row:
        print(cell.value)
"""

"\ndata = 'ABCDEFGHI'\nfor char in data:\n    val = char + '1'\n    print(ws[val].value)\n\nprint(ws.cell(row=1, column=2).value)\n\n#Iterate through columns of the first row\nfor row in ws.iter_rows(min_row=1, max_row=1):\n    for cell in row:\n        print(cell.value)\n"

In [15]:
from exchangelib import Account, Message, Credentials, HTMLBody
from exchangelib import Configuration, DELEGATE
import os

In [16]:
outlook_user = os.environ.get('OUTLOOK_USER')
outlook_password = os.environ.get('OUTLOOK_PASS')
outlook_server = os.environ.get('OUTLOOK_SERVER')
outlook_email = os.environ.get('OUTLOOK_EMAIL')

print(os.getenv('OUTLOOK_USER'))
print(os.getenv('OUTLOOK_PASS'))

ioc\00504802
Toptal@22


In [17]:
credentials = Credentials(username=outlook_user,
                         password=outlook_password
                         )
config = Configuration(server=outlook_server,
                      credentials=credentials)
account = Account(primary_smtp_address=outlook_email,
                 config=config,
                 autodiscover=False,
                 access_type=DELEGATE)

In [18]:
def create_html(ws):
    html = """
    <html><body><h3>Nozzle Sales Report:</h3>
    <table border=1>
    """
    # entering header data in html
    for whole_row in ws.iter_rows(min_row=1, max_row=2):
        html += "<tr>"
        for row in whole_row:
            html += f'<th>{row.value}</th>'
        html += "</tr>"
    # entering table data in html
    for whole_row in ws.iter_rows(min_row=3):
        html += "<tr>"
        for row in whole_row:
            if isinstance(row.value, float):
                html += f'<td>{round(row.value, 2)}</td>'
                print(round(row.value, 2))
            else:
                html += f'<td>{row.value}</td>'
        html += "</tr>"
    html += """
    </table>
    <p>
    <br>
    With Regards<br>
    Pankaj Barnwal<br>
    </p>
    </body>
    </html>
    """
    return html

In [19]:
msg = Message(
    account=account,
    subject="Nozzle Sales Report - Sambalpur DO",
    body=HTMLBody(create_html(ws)),
    to_recipients=['barnwalp@indianoil.in']
)

msg.send_and_save()

48173.73
49982.35
38821.91
48629.88
43583.87
50826.26
32889.11
25587.9
42687.11
44234.87
47192.44
53251.3
49073.05
54930.81
39575.28
31448.45
76242.47
50408.61
68978.65
77747.53
72591.25
50165.91
38469.76
154855.57
42621.4
126889.83
42844.78
110250.46
48242.24
72423.87
35329.92
274345.39
48917.65
275982.19
54264.92
286636.3
59103.49
260915.45
39056.62
88873.02
38263.36
72204.61
39106.79
73976.58
35928.13
65449.13
19537.97
73249.58
17943.48
81982.21
24699.31
104697.34
29944.66
81805.33
17257.64
8608.11
6954.99
8533.56
9627.54
8246.35
9651.46
3594.88
6057.07
